In [1]:
import Base: ==, isempty,⊆, ∪, ∩, ∈, ∉, +, -, *, ^, iszero, /, inv

In [2]:
export Intervalo, intervalo_vacio, ⪽, ⊔, division_extendida

In [3]:
struct Intervalo{T<:Real}
    infimo::T
    supremo::T
    
    function Intervalo(infimo,supremo)
        infimo1, supremo1, _ = promote(infimo, supremo, 1.0) 
        infimo1 > supremo1  ? error("Supremo es menor que el ínfimo") : new{typeof(infimo1)}(infimo1,supremo1)
    end
end

In [4]:
Intervalo(infimo) = Intervalo(infimo,infimo) 

Intervalo

In [5]:
function intervalo_vacio(T::Type)
        return Intervalo(T(NaN))
end
    
function intervalo_vacio(a::Intervalo{T}) where T <: Real
    return intervalo_vacio(T)
end

intervalo_vacio (generic function with 2 methods)

In [6]:
function isempty(a::Intervalo)
    isnan(a.infimo) && isnan(a.supremo) && return true
    return false 
end

isempty (generic function with 39 methods)

In [7]:
function ==(a::Intervalo, b::Intervalo)
    isempty(a) && isempty(b) && return true
    a.infimo == b.infimo && a.supremo == b.supremo
end

== (generic function with 172 methods)

In [8]:
function ⊆(a::Intervalo, b::Intervalo)
    isempty(a) && return true
    b.infimo ≤ a.infimo && a.supremo ≤ b.supremo
end

issubset (generic function with 5 methods)

In [9]:
function ⪽(a::Intervalo, b::Intervalo)
    isempty(a) && return true
    b.infimo < a.infimo && a.supremo < b.supremo
end

⪽ (generic function with 1 method)

In [10]:
function ∪(a::Intervalo, b::Intervalo)
    isempty(a) && return(b)
    isempty(b) && return(a)
    return Intervalo(min(a.infimo,b.infimo),max(a.supremo,b.supremo))
end

union (generic function with 6 methods)

In [11]:
function ⊔(a::Intervalo, b::Intervalo)
    return a∪b 
end

⊔ (generic function with 1 method)

In [12]:
function ∩(a::Intervalo, b::Intervalo)
    isempty(a) && return a
    isempty(b) && return b
    b.supremo < a.infimo && return Intervalo(NaN)
    a.supremo < b.infimo && return Intervalo(NaN)
    return Intervalo(max(a.infimo,b.infimo),min(a.supremo,b.supremo))
end

intersect (generic function with 23 methods)

In [13]:
function ∈(a::Real, b::Intervalo)
    isinf(a) && return false
    b.infimo ≤ a ≤ b.supremo
end 

in (generic function with 37 methods)

In [14]:
function +(a::Intervalo, b::Intervalo)
    return Intervalo(prevfloat(a.infimo + b.infimo), nextfloat(a.supremo + b.supremo))
end
    
function +(a::Real, b::Intervalo)
    return Intervalo(a) + b
end
    
function +(a::Intervalo, b::Real)
    return a + Intervalo(b)
end

+ (generic function with 209 methods)

In [15]:
function -(a::Intervalo, b::Intervalo)
    return Intervalo(prevfloat(a.infimo - b.supremo), nextfloat(a.supremo - b.infimo))
end
    
function -(a::Real, b::Intervalo)
    return Intervalo(a) - b
end
    
function -(a::Intervalo, b::Real)
    return a - Intervalo(b)
end
    
function -(a::Intervalo)
    return Intervalo(-a.supremo, -a.infimo)
end

- (generic function with 215 methods)

In [16]:
function iszero(a::Intervalo)
    return a == Intervalo(0.0)
end

iszero (generic function with 15 methods)

In [17]:
function *(a::Intervalo, b::Intervalo)
    (isempty(a) || isempty(b)) && return Intervalo(NaN)
    (iszero(a) || iszero(b)) && return Intervalo(0.0)    
    l1 = (prevfloat(a.infimo*b.infimo), prevfloat(a.infimo*b.supremo), prevfloat(a.supremo*b.infimo), prevfloat(a.supremo*b.supremo))
    l2 = (nextfloat(a.infimo*b.infimo), nextfloat(a.infimo*b.supremo), nextfloat(a.supremo*b.infimo), nextfloat(a.supremo*b.supremo))
    return Intervalo(minimum(l1),maximum(l2))
end

function *(a::Real, b::Intervalo)
    return Intervalo(prevfloat(a*b.infimo), nextfloat(a*b.supremo))
end
    
function *(a::Intervalo, b::Real)
    return Intervalo(prevfloat(a.infimo*b), nextfloat(a.supremo*b))
end

* (generic function with 323 methods)

In [18]:
function ^(a::Intervalo, b::Int)
    b < 0 && return inv(a^b)
    b == 1 && return a
    
    c = min(a.infimo^b, a.supremo^b)
    d = max(a.infimo^b, a.supremo^b)
    
    iseven(b) && 0 ∈ a && return Intervalo(0, nextfloat(d))
    iseven(b) && return Intervalo(prevfloat(c), nextfloat(d)) ∩ Intervalo(0, Inf)
    return Intervalo(prevfloat(c), nextfloat(d))  
end

^ (generic function with 69 methods)

In [19]:
function inv(a::Intervalo)
    iszero(a) && return Intervalo(NaN)
    return Intervalo(1.0)/a
end

inv (generic function with 34 methods)

In [20]:
function /(a::Intervalo, b::Intervalo)
    0 ∉ b && return a*Intervalo(1/b.supremo,1/b.infimo)
    b.infimo < 0.0 < b.supremo && return Intervalo(-Inf, Inf)
    0.0 == b.infimo < b.supremo && return Intervalo(prevfloat(min(a.infimo*(1/b.supremo), a.supremo*(1/b.supremo))), Inf)
    b.infimo < b.supremo == 0.0 && return Intervalo(-Inf, nextfloat(max(a.infimo*(1/b.infimo), a.supremo*(1/b.infimo))))       
    iszero(b) && return Intervalo(NaN)
end
    
function /(a::Real, b::Intervalo)
    a == 1 && return inv(b)
end

/ (generic function with 140 methods)

In [21]:
function division_extendida(a::Intervalo, b::Intervalo)
    0∉b && return (a/b,)
    0∈a && 0∈b && return (Intervalo(-Inf, Inf),)
    a.supremo < 0 && b.infimo < b.supremo == 0 &&return (Intervalo(a.supremo/b.infimo, Inf),)
    a.supremo < 0 && b.infimo < 0 <  b.supremo && return (Intervalo(-Inf, nextfloat(a.supremo/b.supremo)) , (Intervalo(prevfloat(a.supremo/b.infimo), Inf)))
    a.supremo < 0 && 0 == b.infimo < b.supremo && return (Intervalo(-Inf, a.supremo/b.supremo),)
    0 < a.infimo && b.infimo < b.supremo == 0 && return (Intervalo(-Inf, nextfloat(a.infimo/b.infimo)),)
    0 < a.infimo && b.infimo < 0 < b.supremo && return (Intervalo(-Inf, nextfloat(a.infimo/b.infimo)) , (Intervalo(prevfloat(a.infimo/b.supremo), Inf)))
    0 < a.infimo && 0 == b.infimo < b.supremo && return (Intervalo(prevfloat(a.infimo/b.supremo), Inf),)
    0∉a && iszero(b) && return (Intervalo(NaN),)
end

division_extendida (generic function with 1 method)

In [89]:
using Test

# using Intervalos

@testset "Tarea1" begin

    u = Intervalo(1.0)
    z = Intervalo(0.0)
    a = Intervalo(1.5, 2.5)
    b = Intervalo(1, 3)
    c = Intervalo(BigFloat("0.1"), big(0.1))
    d = Intervalo(-1, 1)
    f = Intervalo(prevfloat(Inf))
    emptyFl = intervalo_vacio(Float64)
    emptyB = intervalo_vacio(BigFloat)
    realesFl = Intervalo(-Inf,Inf)

    @testset "Creación de intervalos" begin
        @test typeof(a) == Intervalo{Float64}
        @test getfield(a, :infimo) == 1.5
        @test getfield(a, :supremo) == 2.5

        @test typeof(b) == Intervalo{Float64}
        @test getfield(b, :infimo) == 1.0
        @test getfield(b, :supremo) == 3.0

        @test typeof(c) == Intervalo{BigFloat}
        @test getfield(c, :infimo) == BigFloat("0.1")
        @test getfield(c, :supremo) == big(0.1)

        @test typeof(emptyFl) == Intervalo{Float64}
        @test typeof(emptyB) == Intervalo{BigFloat}
        @test typeof(realesFl) == Intervalo{Float64}
        @test getfield(realesFl, :supremo) == Inf

        @test u == Intervalo(1.0)
        @test z == Intervalo(0.0)
        @test z == Intervalo(big(0.0))
    end

    @testset "Operaciones de conjuntos" begin
        @test a == a
        @test a !== b
        @test c == c
        @test emptyFl == emptyB
        @test a ⊆ a
        @test a ⊆ b
        @test emptyFl ⊆ b
        @test b ⊇ a
        @test c ⊆ c
        @test !(a ⊆ emptyFl)
        @test !(emptyB ⊇ c)
        @test !(c ⊆ b) && !(b ⊆ c)
        @test a ⪽ b
        @test emptyFl ⪽ b
        @test !(c ⪽ c)
        @test a ∪ b == b
        @test a ⊔ b == b
        @test a ∪ emptyFl == a
        @test c ⊔ emptyB == c
        @test a ∩ b == a
        @test a ∩ emptyFl == emptyFl
        @test a ∩ c == emptyB
        @test 0 ∈ z
        @test Inf ∉ realesFl
        @test Inf ∉ emptyFl
        @test 2 ∈ a
        @test 3 ∉ a
        @test 0.1 ∈ c
    end

    @testset "Operaciones aritméticas" begin
        @test emptyFl + z == emptyFl
        @test z + z == Intervalo(prevfloat(0.0), nextfloat(0.0))
        @test u + z == u + 0.0
        @test z - u == 0.0-u
        @test -u == Intervalo(-u.supremo, -u.infimo)
        @test f + f !== intervalo_vacio(f)
        @test f + f == Intervalo(f.infimo, Inf)
        @test getfield(f + f, :supremo) == Inf
        @test -u ⪽ z - u
        @test b + 1 == 1.0 + b == Intervalo(prevfloat(2.0), nextfloat(4.0))
        @test d ⪽ 2*(a - 2)
        @test c - 0.1 !== z
        @test emptyFl * z == emptyFl
        @test Intervalo(0.1, 0.1) ⪽ 0.1 * u
        @test Intervalo(-0.1, 0.1) ⪽ d * 0.1
        @test d * (a + b) ⊆ d*a + d*b
        @test d^1 == d
        @test d^2 == Intervalo(0, nextfloat(1.0))
        @test emptyB^2 == emptyB
        @test emptyFl^3 == emptyFl
        @test Intervalo(0,Inf) * u == Intervalo(prevfloat(0.0), Inf)
        @test Intervalo(-Inf,Inf) * z == z
        @test emptyFl * z == emptyFl
        @test d ⪽ d*d
        @test d ⪽ d^3
        @test d^4 == d^2
        @test Intervalo(-0.25,0.5)^2 == (-Intervalo(-0.25,0.5))^2 == Intervalo(0.0, 0.5)^2
        @test Intervalo(-0.25,0.5)^3 == -(-Intervalo(-0.25,0.5))^3 == Intervalo(prevfloat(-1/2^6), nextfloat((1/2^3)))
        @test a / emptyFl == emptyFl
        @test emptyB / c == emptyB
        @test emptyFl / emptyFl == emptyFl
        @test u/d == Intervalo(-Inf, Inf)
        @test inv(Intervalo(0.0)) == emptyFl
        @test 0.5 ∈ u/a
        @test 1 ∈ a/a
        @test c^-1 == 1/c
        @test inv(a) == 1/a
        @test (a-1)*(a-2) ⪽ (a*a -3*a + 3)
        @test a/d == Intervalo(-Inf, Inf)
        @test z/z == intervalo_vacio(z)
        @test u ⪽ 1/u
        @test all(division_extendida(u,z) .== (emptyFl,))
        @test all(division_extendida(z,z) .== (Intervalo(-Inf,Inf),))
        @test all(division_extendida(u,u) .== (u/u,))
        @test all(division_extendida(u, Intervalo(0,1)) .== (u/Intervalo(0,1),))
        @test all(division_extendida(u,d) .== (u/Intervalo(-1,0), u/Intervalo(0,1)))
    end

end

Test Summary: | Pass  Total  Time
Tarea1        |   90     90  0.4s


Test.DefaultTestSet("Tarea1", Any[Test.DefaultTestSet("Creación de intervalos", Any[], 16, false, false, true, 1.668043444747e9, 1.668043444747e9), Test.DefaultTestSet("Operaciones de conjuntos", Any[], 28, false, false, true, 1.668043444747e9, 1.668043444748e9), Test.DefaultTestSet("Operaciones aritméticas", Any[], 46, false, false, true, 1.668043444748e9, 1.668043445116e9)], 0, false, false, true, 1.668043444747e9, 1.668043445116e9)